In [12]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd, det
from ipywidgets import interact, interactive, fixed, FloatSlider, Layout, VBox, HBox, HTML
from IPython.display import display

def plot_circle_transform(ax, matrix, color, linestyle='-', label=None):
    t = np.linspace(0, 2*np.pi, 1000)
    circle = np.array([np.cos(t), np.sin(t)])
    transformed = matrix @ circle
    ax.plot(transformed[0], transformed[1], color=color, linestyle=linestyle, label=label)

def update(a, b, c, d):
    A = np.array([[a, b], [c, d]])
    U, s, Vt = svd(A)
    S = np.diag(s)

    fig, axs = plt.subplots(2, 2, figsize=(12, 12))
    axs = axs.ravel()

    for ax in axs:
        ax.set_xlim(-1.5, 1.5)
        ax.set_ylim(-1.5, 1.5)
        ax.set_aspect('equal')
        ax.grid(True)
        ax.add_artist(plt.Circle((0, 0), 1, fill=False, color='gray', linestyle='--'))

    # Original matrix A
    plot_circle_transform(axs[0], A, 'b', label='A')
    axs[0].set_title('Original Transformation (A)')

    # V* (rotation/reflection)
    plot_circle_transform(axs[1], Vt, 'g', label='V*')
    axs[1].set_title('V* (Rotation/Reflection)')

    # Σ (scaling)
    plot_circle_transform(axs[2], S, 'r', label='Σ')
    axs[2].set_title('Σ (Scaling)')

    # U (rotation/reflection)
    plot_circle_transform(axs[3], U, 'm', label='U')
    axs[3].set_title('U (Rotation/Reflection)')

    # Add arrows to show rotation/reflection for U and V*
    for ax, matrix, color in [(axs[1], Vt, 'g'), (axs[3], U, 'm')]:
        ax.arrow(0, 0, matrix[0, 0], matrix[1, 0], head_width=0.05, head_length=0.1, fc=color, ec=color)
        ax.arrow(0, 0, matrix[0, 1], matrix[1, 1], head_width=0.05, head_length=0.1, fc=color, ec=color)

    for ax in axs:
        ax.legend()

    plt.tight_layout()

    # Add text to display matrix and SVD components
    fig.text(0.01, 0.99, f"Matrix A =\n[[{a:.2f}, {b:.2f}]\n [{c:.2f}, {d:.2f}]]", fontsize=10, verticalalignment='top', fontfamily='monospace')
    fig.text(0.01, 0.85, f"U =\n{U[0,0]:.2f}  {U[0,1]:.2f}\n{U[1,0]:.2f}  {U[1,1]:.2f}", fontsize=10, verticalalignment='top', fontfamily='monospace')
    fig.text(0.01, 0.71, f"Σ =\n{s[0]:.2f}  0\n0  {s[1]:.2f}", fontsize=10, verticalalignment='top', fontfamily='monospace')
    fig.text(0.01, 0.57, f"V* =\n{Vt[0,0]:.2f}  {Vt[0,1]:.2f}\n{Vt[1,0]:.2f}  {Vt[1,1]:.2f}", fontsize=10, verticalalignment='top', fontfamily='monospace')
    fig.text(0.01, 0.43, f"det(A) = {det(A):.2e}", fontsize=10, verticalalignment='top', fontfamily='monospace')

    plt.show()

# Create interactive widget
interactive_plot = interactive(update,
                               a=FloatSlider(min=-2, max=2, step=0.1, value=1, description='A'),
                               b=FloatSlider(min=-2, max=2, step=0.1, value=0, description='B'),
                               c=FloatSlider(min=-2, max=2, step=0.1, value=0, description='C'),
                               d=FloatSlider(min=-2, max=2, step=0.1, value=1, description='D'))

# Create description
description = HTML("""
<h3>Interactive Singular Value Decomposition (SVD) Visualization</h3>
<p>This interactive plot demonstrates the geometric interpretation of the SVD for a 2x2 matrix A.</p>
<h4>How to use:</h4>
<ul>
    <li>Use the sliders to adjust the values of matrix A = [[A, B], [C, D]].</li>
    <li>Observe how changing these values affects the transformation of the unit circle and the components of the SVD.</li>
</ul>
<h4>What you're seeing:</h4>
<ul>
    <li><strong>Top-left:</strong> Transformation by the original matrix A</li>
    <li><strong>Top-right:</strong> Effect of V* (rotation/reflection)</li>
    <li><strong>Bottom-left:</strong> Effect of Σ (scaling)</li>
    <li><strong>Bottom-right:</strong> Effect of U (rotation/reflection)</li>
</ul>
<h4>Mathematical interpretation:</h4>
<p>For a matrix A, the SVD is given by A = UΣV*, where:</p>
<ul>
    <li>U and V* are unitary matrices (rotations/reflections)</li>
    <li>Σ is a diagonal matrix with singular values (scaling factors)</li>
</ul>
<p>The plots show how these individual components transform the unit circle, compared to the full transformation by A.</p>
<h4>Displayed information:</h4>
<ul>
    <li>Current matrix A values and its determinant are shown on the left side of the plot.</li>
    <li>The resulting U, Σ, and V* matrices from the SVD are displayed on the left side of the plot.</li>
</ul>
<p><strong>Note:</strong> The gray dashed circle in each plot represents the original unit circle for reference. Arrows in the U and V* plots show the direction of the basis vectors after transformation.</p>
""")

# Display the description and interactive plot
display(VBox([description, interactive_plot]))